## Defining a Classification Problem

The following section explores some ideas of turning stock price predictions into a classification ML problem. Namely, we will explore two questions:

1) Can we tell if a stock will go up or down tomorrow? (It turns out, this is quite difficult)

2) Can we tell if a stock is "trending up" or "trending down"? (This shows a bit of optimism.

Our approach is as follows:

1a) If _perfectly_ executed (that is, if we had actual future data), how much money would this strategy make compared to just buying and holding a stock? We do some backtesting to get a benchmark for the efficacy of the approach.

1b) How realistic is it to predict such movement? We discover that a less-lucrative approach (when executed "perfectly") might be better in practice due to our ability to actually predict it.

2) Using 69 well established technical indicators for trend, momentum, volatility, and volume, can we predict the movement of a stock? How correlated are these indicators? How useful are all of these features in our prediction? We do a correlation analysis and a Random Forest model to assess.

3) Could we do better (albeit, less clearly) by creating the same number of features as selected in step 2 as "fake features" by doing a PCA on the original 69 indicators? Can we improve the accuracy of our top features by adding an element of non-linearity using the Kernel trick?

4) How can we account for the timeseries nature of stocks? We explore adding lag variables to our feature matrix. We also explored "slopes" using linear regression but they were not particularly helpful and have been dropped from the final presentation.

5) Finally, we take a look at a non-linear approach -- the Multi-Layer Perceptron. With the inclusion of the lag variables and two hidden layers, we achieve promising results (>90%) for predicting our chosen indicator. This leaves us with a sense of optimism and motivation for further exploration. We would next investigate engineering more time-related features, as well as using a Recurrent Neural Networks which is well suited for time series data.

### Support Clases
We utilize the following classes in these sections to help back test, create labels, and engineer features:

In [ ]:
import ta


class StockTechnicals:
    """
    A class for creating technical indicators of stocks as features and labels from various strategies for training
    Machine Learning models
    :param data: A pandas dataframe of a daily stock ticker data
    Usage:
    a = StockTechnicals(data)
    X = a.features
    y = a.price_will_rise()
    """

    def __init__(self, data):
        self.data = data
        self._features = ta.add_all_ta_features(
            self.data,
            open="Open",
            high="High",
            low="Low",
            close="Close",
            volume="Volume"
        )

    @property
    def features(self):
        return self._features.drop(
            columns=[
                "Date",
                "Open",
                "High",
                "Low",
                "Close",
                "Adj Close",
                "Volume",
                'trend_psar_up',
                'trend_psar_down',
                'trend_psar']
        ).dropna()

    # Can add more derived metrics here
    # e.g. slopes, cross-overs
    def add_slopes(self, metrics=(), days=()):
        if not metrics:
            metrics = list(self.features)
        for m in metrics:
            for d in days:
                self._features[f"{m}_{d}_day_slope"] = self._features.apply(self._add_slope, metric=m, days=d, axis=1)

    def _add_slope(self, row, metric, days=3):
        if row.name < days + 1:
            return None
        x = np.arange(days)
        A = np.vstack([x, np.ones(len(x))]).T
        y = self._features.loc[row.name - days:row.name - 1, metric].values
        slope, _ = np.linalg.lstsq(A, y, rcond=None)[0]
        return slope

    def add_lags(self, metrics=(), days=()):
        if not metrics:
            metrics = list(self.features)
        for m in metrics:
            for d in days:
                self._features[f"{m}_{d}_day_lag"] = self._features.apply(self._add_lag, metric=m, days=d, axis=1)

    def _add_lag(self, row, metric, days=3):
        if row.name < days + 1:
            return None
        return self._features.loc[row.name - days, metric]

    # Possible Strategy labels

    def price_will_rise(self, days=1):
        return np.array(self.features.apply(self._price_will_rise, days=days, axis=1))

    def _price_will_rise(self, row, days=1):
        try:
            if self.data.loc[row.name + days, 'Adj Close'] > self.data.loc[row.name, 'Adj Close']:
                return 1
            return 0
        except KeyError:
            return None

    def future_sma_higher_than_current_price(self, days=7):
        # This metric is the most promising as far as an investment strategy
        return np.array(self.features.apply(self._future_sma_v_price, days=days, axis=1))

    def _future_sma_v_price(self, row, days=7):
        try:
            future_sma = np.mean([self.data.loc[row.name + day + 1, 'Adj Close'] for day in range(days)])
        except KeyError:
            return None

        try:
            if future_sma > self.data.loc[row.name, 'Adj Close']:
                return 1
            return 0
        except KeyError:
            return None

In [ ]:
from pyalgotrade import strategy


class TradeHoldStrategy(strategy.BacktestingStrategy):
    """
    A class to test strategies based on a feed of signals (1's and 0's)
    The strategy buys on next 1 and holds until next 0, when it sells and repeats.
    :param future_signals: numpy array of signals of the same length as the feed on which to trade
    """

    def __init__(self, feed, instrument, future_signals, verbose=False):
        strategy.BacktestingStrategy.__init__(self, feed)
        self.instrument = instrument
        self.future_signals = future_signals
        self.adj_close = feed[instrument].getAdjCloseDataSeries()
        self.day = 0
        self.position = None
        self.verbose = verbose

    def onBars(self, bars):
        try:
            todays_signal = self.future_signals[self.day]
            if self.day == 0:
                yesterdays_signal = 0
            else:
                yesterdays_signal = self.future_signals[self.day - 1]

            if todays_signal > yesterdays_signal:
                self.position = self.enterLong(self.instrument, 100)
            elif todays_signal < yesterdays_signal:
                self.position.exitMarket()
        except IndexError:
            pass
        self.day += 1

    def onEnterOk(self, position):
        if self.verbose:
            exec_info = position.getEntryOrder().getExecutionInfo()
            self.info("BUY at $%.2f" % (exec_info.getPrice()))

    def onExitOk(self, position):
        if self.verbose:
            exec_info = position.getExitOrder().getExecutionInfo()
            self.info("SELL at $%.2f" % (exec_info.getPrice()))
        self.position = None

### Picking a Strategy as a Classificaion Problem

In [ ]:
# The following code backtests some possible "returns" when implementing a strategy using the 
# actual future data to generate signals. We use the convention TradeHoldStrategy where we are 
# attempting to find a signal (a classification) to tell us to buy, hold or sell.
# It is our goal to find one that proves to make money and that we can somewhat accurately predict.

from copy import deepcopy
import pandas as pd
import numpy as np

from src.features.build_features import StockTechnicals

from src.models.backtest_strategy import TradeHoldStrategy

In [ ]:
# load data using MSFT 
ticker = "MSFT"
all_daily_data = pd.read_csv(f'../data/{ticker}.csv')

In [ ]:
# create a feature matrix and some labels using our handy StockTechnicals class
# we will actually only need labels for this decision, as we're using observed data
technicals = StockTechnicals(all_daily_data)
X = technicals.features
y = technicals.price_will_rise()

In [ ]:
### BACKTESTING ###
# import a backtesting library
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.stratanalyzer import returns, trades

In [ ]:
# build our backtesing feed using MSFT daily data
base_feed = yahoofeed.Feed()
base_feed.addBarsFromCSV(f"{ticker}", f'../data/{ticker}.csv')

In [ ]:
# As a benchmark, we will buy 100 shares of MSFT on day one and hold. 

In [ ]:
# buy one share on day one and hold as a benchmark
benchmark_feed = deepcopy(base_feed)
benchmark_trades = np.ones(len(all_daily_data))
benchmark_strat = TradeHoldStrategy(benchmark_feed, f'{ticker}', benchmark_trades)

# performance metrics for strategy evaluation
ret_analyzer = returns.Returns()
benchmark_strat.attachAnalyzer(ret_analyzer)
tradesAnalyzer = trades.Trades()
benchmark_strat.attachAnalyzer(tradesAnalyzer)

In [ ]:
# backtest the strategy 
benchmark_strat.run()
bmk_value = round(benchmark_strat.getResult() - 1000000, 2)
print(f"Final portfolio increase: ${bmk_value}")
print(f"Total trades: {tradesAnalyzer.getCount()}")

In [ ]:
# Now let's backtest a strategy with the training labels we created in StockTechnicals. The 
# strategy is to BUY 100 shares if we predict the price will rise the following day and HOLD 
# until we predict the market will go down the following day, when we SELL 100 shares and 
# wait until we predict another rise.
# Note that there is some loss as we can't trade after-hours in our sim and open prices do not 
# always match closing prices, but it is still very good (results below)

In [ ]:
perf_feed = deepcopy(base_feed)
perf_trades = np.concatenate([np.zeros(len(all_daily_data) - len(y)), y])
perf_strat = TradeHoldStrategy(perf_feed, f'{ticker}', perf_trades)

# performance metrics
ret_analyzer = returns.Returns()
perf_strat.attachAnalyzer(ret_analyzer)
tradesAnalyzer = trades.Trades()
perf_strat.attachAnalyzer(tradesAnalyzer)

In [ ]:
# backtest the strategy
perf_strat.run()
perf_strat_value = round(perf_strat.getResult() - 1000000, 2)
print(f"Final portfolio increase: ${perf_strat_value}")
perf_pct_improve = round((perf_strat_value / bmk_value - 1) * 100, 1)
print(f"Percentage gain v. buy-and-hold bmk: {perf_pct_improve}%")
print("Total trades: %d" % (tradesAnalyzer.getCount()))

In [ ]:
# As we can see, perfect execution of this strategy would improve our returns by over 400%
# However daily stock movement is notoriously difficult to predict, so we'll explore another
# strategy as well.

In [ ]:
# Backtest another strategy. This strategy is to to BUY 100 shares if we predict the N-day SMA 
# will be higher than the current stock price in N days, and HOLD until we predict the N-day SMA 
# will be lower than the current stock price in N days, when we SELL 100 shares and wait until 
# we predict another rise.
N = 26

# generate our labels using our StockTechnicals class
y_sma = technicals.future_sma_higher_than_current_price(days=N)

sma_feed = deepcopy(base_feed)
sma_trades = np.concatenate([np.zeros(len(all_daily_data) - len(y_sma)), y_sma])
sma_strat = TradeHoldStrategy(sma_feed, f'{ticker}', sma_trades)

# performance metrics
ret_analyzer = returns.Returns()
sma_strat.attachAnalyzer(ret_analyzer)
tradesAnalyzer = trades.Trades()
sma_strat.attachAnalyzer(tradesAnalyzer)

In [ ]:
# run the N-day sma strategy
sma_strat.run()
sma_value = round(sma_strat.getResult() - 1000000, 2)
print(f"Final portfolio increase: ${sma_value}")
sma_pct_improve = round((sma_value / bmk_value - 1) * 100, 1)
print(f"Percentage gain v. buy-and-hold bmk: {sma_pct_improve}%")
print("Total trades: %d" % (tradesAnalyzer.getCount()))

In [ ]:
# As we can see with this strategy, the potential returns are not as high, but 
# it's more conservative and likely a bit easier to predict as it's using a more general metrics
# in the moving average rather than daily close data. We will demonstrate this next.

In [ ]:
### EYEBALLING FOR FEASIBILITY ###
# Using a naive, unoptimized Logistic Regression model, we will evaluate benchmark peformances for 
# each of the two strategies outlined above.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# TEST NEXT DAY STRATEGY IN LOGISTIC REGRESSION

# set up X_daily and y_daily (y has trailing nan values, so we truncate here) 
y_daily = y[~np.isnan(y)]
X_daily = X[:len(y_daily)]

# preprocess the data
X_train, X_test, y_train, y_test = train_test_split(X_daily, y_daily, random_state=2, stratify=y_daily)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# run a logistic regression
for c in [0.01, 0.1, 1.0, 10, 100]:
    lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
    lr.fit(X_train_std, y_train)
    y_pred = lr.predict(X_test_std)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")

In [ ]:
# As we can see above, a naive LogisticRegression doesn't do any better than simply guessing
# whether the stock will rise the following day

In [ ]:
# TEST N-DAY SMA STRATEGY IN LOGISTIC REGRESSION

y_sma = y_sma[~np.isnan(y_sma)]
X_sma = X[:len(y_sma)]

# preprocess the data
X_train, X_test, y_train, y_test = train_test_split(X_sma, y_sma, random_state=2, stratify=y_sma)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# run a logistic regression
for c in [0.01, 0.1, 1.0, 10, 100]:
    lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
    lr.fit(X_train_std, y_train)
    y_pred = lr.predict(X_test_std)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Compared to predicting the one-day return, predicting the 26-day sma rise is much more
# promising. Our _very_ naive Logistic Regression classifer scores at almost 64%. For this
# reason, we will move forward attempting to predict these labels.

### Feature Selection using Correlations and Random Forests

In [ ]:
from copy import deepcopy
import pandas as pd
import numpy as np

from src.features.build_features import StockTechnicals

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# load data using MSFT 
ticker = "MSFT"
all_daily_data = pd.read_csv(f'../data/{ticker}.csv')

In [ ]:
# set up our X and y

N = 26

# create a feature matrix and some labels using our handy StockTechnicals class
technicals = StockTechnicals(all_daily_data)
X = technicals.features
y = technicals.future_sma_higher_than_current_price(N)

# we don't have the last N days of data
y = y[~np.isnan(y)]
X = X[:len(y)]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, stratify=y)


In [ ]:
# take a look at the relative importances of all features using a random forest 
feat_labels = list(X.columns)
forest = RandomForestClassifier(n_estimators=500, random_state=1)
forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
    print(f"{f + 1:2}) {X.columns[indices[f]]:<28} {round(importances[indices[f]], 4):>}")

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# check the top 20 features to see if any are correlated
top_feats_X = X_train.iloc[:,indices[:20]]
corr = top_feats_X.corr()
plt.figure(figsize=(12, 12))
sns.heatmap(corr, annot = True, label = ticker)

In [ ]:
# there are lots of correlated features, so let's remove them in order of priority
ind_feats = []
for i in indices:
    # iterating in importance order, don't include a feature if it has a correlation
    # stronger than 0.9 (or -0.9) with a features already included
    if all(abs(X_train.corr().iloc[ind_feats, i]) < 0.9):
        ind_feats.append(i)

print(f'We are left with {len(ind_feats)} "independent" (non-correlated) features')

In [ ]:
# observe feature importances of this subset using a new random forest
ind_feats_X = X_train.iloc[:, ind_feats]

ind_feats_forest = RandomForestClassifier(n_estimators=500, random_state=1)
ind_feats_forest.fit(ind_feats_X, y_train)
ind_importances = ind_feats_forest.feature_importances_

for rk, (idx, imp) in enumerate(sorted(zip(ind_feats, ind_importances), key=lambda x: x[1], reverse=True)):
    print(f"{rk + 1:2}) {X.columns[idx]:<28} {round(imp, 4):>}")

In [ ]:
# Setting a threshold of 0.3 leaves us with 13 indicators as featres, which 
# sounds like a good place to start

In [ ]:
# Take a look again at the correlation matrix of our top 13 features to make sure nothing changed
top_feats_X = X_train.iloc[:,ind_feats[:13]]
corr = top_feats_X.corr()
plt.figure(figsize=(12, 12))
sns.heatmap(corr, annot = True, label = ticker)


### Exploring PCA and KPCA approaches

In [ ]:
import numpy as np
import pandas as pd

from sklearn.decomposition import KernelPCA, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

from src.features.build_features import StockTechnicals

In [ ]:
# Load training data using MSFT
train_ticker = "MSFT"
all_msft_data = pd.read_csv(f'../data/{train_ticker}.csv')

# number of future days for SMA to rise
N = 26

In [ ]:
technicals = StockTechnicals(all_msft_data)
X = technicals.features
y = technicals.future_sma_higher_than_current_price(N)

# we don't have the last N days of data (they're in the future)
y = y[~np.isnan(y)]
X = X[:len(y)]

In [ ]:
# limit to top features as determined by a RandomForest in feature-selection.ipnyb
top_features = [
    'trend_visual_ichimoku_b',
    'volume_obv',
    'volatility_kcw',
    'volatility_atr',
    'trend_mass_index',
    'trend_kst_sig',
    'volume_cmf',
    'trend_adx',
    'trend_macd_signal',
    'volatility_bbw',
    'trend_kst_diff',
    'momentum_tsi',
    'trend_macd_diff',
]

In [ ]:
# limit X to top features only
X_top = X[top_features]
X_train, X_test, y_train, y_test = train_test_split(X_top, y, random_state=2, stratify=y)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
# run a logistic regression to get a benchmark performance on the top 13 features
for c in [0.01, 0.1, 1.0, 10, 100]:
    lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
    lr.fit(X_train_std, y_train)
    y_pred = lr.predict(X_test_std)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}\n"
          f"F1-Score:       {f1_score(y_test, y_pred)}")
    

In [ ]:
# Limiting to the top 13 features achieves an accuracy very close to using all 69 (~65% v ~69%).
# As an alternative, let's see what happens when reducing to 13 "fake" features using PCA 

In [ ]:
# set up test and train datasets using full X
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, stratify=y)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# fit a pca with 13 "fake" attributes
pca = PCA(13, random_state=1, tol=0.1)
X_train_std_pca = pca.fit_transform(X_train_std)
X_test_std_pca = pca.transform(X_test_std)

In [ ]:
# run a logistic regression with the 13-feature pca dataset
for c in [0.01, 0.1, 1.0, 10, 100]:
    lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
    lr.fit(X_train_std_pca, y_train)
    y_pred = lr.predict(X_test_std_pca)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}\n"
          f"F1-Score:       {f1_score(y_test, y_pred)}")

In [ ]:
# Using a PCA with 13 "fake" attributes doesn't achieve better results than simply using
# our 13 top attributes as selected from the RandomForest. Therefore, we'll stick with those
# as they're more easily explained. Next we'll try using the kernel trick to see if we can
# improve classification by temporarily projecting data into a higher dimensional space

In [ ]:
# trying out kpca

for k in ('poly', 'rbf', 'sigmoid', 'cosine', 'precomputed'):
    print(f"\nKernel: {k}")
    kpca = KernelPCA(n_components=13, random_state=1, kernel='poly', gamma=1.0)
    X_train_std_kpca = kpca.fit_transform(X_train_std)
    X_test_std_kpca = kpca.transform(X_test_std)
    
    # run a logistic regression
    for c in [0.01, 0.1, 1.0, 10, 100]:
        lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
        lr.fit(X_train_std_kpca, y_train)
        y_pred = lr.predict(X_test_std_kpca)
    
        print(f"Accuracy score: {accuracy_score(y_test, y_pred)}\n"
              f"F1-Score:       {f1_score(y_test, y_pred)}")

In [ ]:
# It doesn't seem like the kernel trick with KPCA is very effective at improving our accuracy either.
# Above is just a sample of the various hyperparameter combinations we attempted. It is our belief
# that we will need to engineer additional features or implement deeper neural networks to improve 
# our classification accuracy

### Adding Lag Variables

In [ ]:
# Stocks are inherently a time-series prediction, so let's see if adding lag values to  various
# metrics helps improve the prediction accuracy using a linear model at all

lag_days = [1, 2, 3, 4, 5, 10, 20]

# add lags is a convenience function I wrote to add trailing values to a list of metrics
all_msft_data = pd.read_csv(f'../data/{train_ticker}.csv')
technicals = StockTechnicals(all_msft_data)
technicals.add_lags(metrics=top_features, days=lag_days)
X = technicals.features
y = technicals.future_sma_higher_than_current_price(26)

# drop final 26 days which we don't have a y value
y = y[~np.isnan(y)]
X = X[:len(y)]

In [ ]:
# limit to top features as determined by a RandomForest in feature-selection.ipnyb
top_features = [
    'trend_visual_ichimoku_b',
    'volume_obv',
    'volatility_kcw',
    'volatility_atr',
    'trend_mass_index',
    'trend_kst_sig',
    'volume_cmf',
    'trend_adx',
    'trend_macd_signal',
    'volatility_bbw',
    'trend_kst_diff',
    'momentum_tsi',
    'trend_macd_diff',
]

top_features.extend([f"{feat}_{n}_day_lag" for feat in top_features for n in lag_days])
X_top = X[top_features]
X_train, X_test, y_train, y_test = train_test_split(X_top, y, random_state=2, stratify=y)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# run a logistic regression
for c in [0.01, 0.1, 1.0, 10, 100]:
    lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
    lr.fit(X_train_std, y_train)
    y_pred = lr.predict(X_test_std)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")

In [ ]:
# Even with lag variables it seems the data just aren't linearly separable.
# Let's try a non-linear classifier like a multi-layer perceptron


### Training a Multi-Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

for a in [0.001, 0.01, 0.1]:
    mlp = MLPClassifier(
        hidden_layer_sizes=(39, 39), 
        solver='lbfgs', 
        max_iter=5000, 
        random_state=2, 
        alpha=a, 
        activation='relu')
    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_test_std)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")

In [ ]:
# There are some early signs of optimism using the MLP classifier with
# predictions as high as 93.6% accuracy with an alpha of 0.01. 39 units
# two hidden layers was selected somewhat arbitrarily as 3x13 (# of impt features)
# Other combinations - larger and smaller - were only nominally different and
# often worse, so this is merely to demonstrate a more "successful" case. 
# Let's try some other hyperparameter combinations for fun.

In [ ]:
for solver in ['adam', 'sgd']:
    for activation in ['relu', 'tanh', 'logistic']:
        mlp = MLPClassifier(
            hidden_layer_sizes=(39, 39), 
            solver=solver,  
            max_iter=1000, 
            random_state=2, 
            alpha=0.01, 
            activation=activation,
            learning_rate='adaptive')
        mlp.fit(X_train_std, y_train)
        y_pred = mlp.predict(X_test_std)

        print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")
        

In [ ]:
# Nothing in the above supercedes our initial MLP (this was an intentional demonstration)
# Perhaps there's a chance we can predict the general "trend" of a stock somewhat accurately 
# with an MLP classifier on a handful of technical indicators along with some lag values.
# Future analysis will be performed on engineering new features as well as recurrent nueral
# network implementations, which are particularly well suited for timeseries data.